# Prelude

With this notebook you should have a number of files:

1. `graph.py`, which contains some class definitions and helper functions (*take a look at it when you start writing your algorithm*).

2. `grass_fire.py` contains an implementation of the Grass Fire algorithm, against the interface defined in the `graph.py` to give you an idea of how things work.

3. `main.py` is used to manually (i.e. not in the notebook) run algorithms and visualize things.

4. `jupy_main.py` defines some high level functions used throughout the book.

5. `common.py` defines some functions used by both `main.py` and `jupy_main.py`.

6. `requirements.txt` to install whatever (numpy and pygame). Numpy is only used for the rng, pygame for the rendering. Neither of those is required, but you'll have to change the code yourself in that case.


# Introduction

In this notebook we will walk you through implementing a simple A* algorithm for path planning.

We have defined an abstract class for a path planning algorithm called `Graph`<sup>[1]</sup>, as well as an abstract class `Node`. Our `main` uses implementations of this interface to show the result of path planning (displaying the path and giving us its cost based on the algorithm's cost function). Your first task should be to take a look at the constructor and members of those two classes (you can ignore the helper function if you wish to).

An example implementation for a planner algorithm has been in `grass_fire.py`. You can try it out:

In [1]:
import grass_fire
grass_fire.main()

Node cost map:
1 1 1 X X X X X X 1
1 1 1 X X X X X X 1
1 1 1 X X X X X X 1
1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1
Total cost-to-go map, initial point at (1, 2)
 3.0  2.0  3.0    ?    ?    ?    ?    ?    ? 12.0
 2.0  1.0  2.0    ?    ?    ?    ?    ?    ? 11.0
 1.0    0  1.0    ?    ?    ?    ?    ?    ? 10.0
 2.0  1.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0
 3.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0 10.0
Path from initial point (1, 2) to goal point (9, 0)
   X    X    X    X    X    X    X    X    X   12
   X    X    X    X    X    X    X    X    X   11
   X    0    X    X    X    X    X    X    X   10
   X    1    2    3    4    5    6    7    8    9
   X    X    X    X    X    X    X    X    X    X


However you will quickly realize that this algorithm scales very poorly to problems with bigger state space:

In [2]:
import jupy_main as jm

jm.run_grass_fire({'grid_size': 5, 'number_obstacles': 0, 'no_render': True})
jm.run_grass_fire({'grid_size': 10, 'number_obstacles': 0, 'no_render': True})
jm.run_grass_fire({'grid_size': 20, 'number_obstacles': 0, 'no_render': True})
jm.run_grass_fire({'grid_size': 50, 'number_obstacles': 0, 'no_render': True})
jm.run_grass_fire({'grid_size': 100, 'number_obstacles': 0, 'no_render': True})
jm.run_grass_fire({'grid_size': 200, 'number_obstacles': 0, 'no_render': True})
jm.run_grass_fire({'grid_size': 400, 'number_obstacles': 0, 'no_render': True})
# Those two are commented because they run for quite some time, feel free to try them out
# around 7s for this one
#jm.run_grass_fire({'grid_size': 500, 'number_obstacles': 0, 'no_render': True})
# ... and 1min for this one
#jm.run_grass_fire({'grid_size': 1000, 'number_obstacles': 0, 'no_render': True})

Path search took 0.512048998643877ms, final cost: 8.0
Path search took 2.0515989999694284ms, final cost: 18.0
Path search took 3.2846399990376085ms, final cost: 38.0
Path search took 16.513803002453642ms, final cost: 98.0
Path search took 102.4066780009889ms, final cost: 198.0
Path search took 551.0734910021711ms, final cost: 398.0
Path search took 3736.1029630010307ms, final cost: 798.0


### Question 1: complexity

a. Given that our map is a square, what is the size of the input `n` based on the size of the grid `m`?

**<font color='red'>$n = m^2$ - the input size `n` (the size of the graph) grows quadratically w.r.t `m`</font>**

b. What seems to be the complexity of this implementation? Please note that we are lookin at `O(n)` and *not* `O(m)`.

<font color='red'>**The complexity seems to be $O(n^2 \cdot ln(n))$. To find this out, notice that, for each iteration with an input size $n = m^2$ and runtime $r(n)$:**
    $$
    \frac{r(n)}{n * ln(n)} = K,~\forall n
    $$
**The intuition behind this result is that the complexity is obviously worse than linear ($O(n)$), but better than quadratic ($O(n^2)$).**
</font>

c. Reading the source of this code (file `grass_fire.py`, function `GrassFire::compute_path`), where is the bottleneck? How could this bottleneck be optimized? What would the complexity be without this bottleneck?

**<font color='red'>To make sure we process the nodes with the lowest cost every iteration (if not, we may explore up to $n^3$ if I'm not mistaken), we sort the list of nodes to explore. Sorting a list takes $O(n \cdot ln(n))$ on the general case. Using a binary heap could reduce the insertion + sorting to a $O(1)$ operation.**
</font>
<font color='red'>  
 **Regarding the new complexity: we explore nodes in increasing cost, and the cost-to-go depends on the destination node only. Those two facts mean that when we set the cost of a node, this cost is guaranteed to be minimum. Since nodes are inserted to our heap only when their cost is reduced, and the cost is reduced exactly once for each node (the first time we process them), we add exactly `n` nodes to the list, and process them exactly once. The new complexity is therefore is $O(n)$.</font>**



# Reminder: the algorithm description

<font color='blue'>
TODO: make sure those guys know wassup
</font>


# The beginning: our graph's Node class

For this exercise we give you already the definition of the `Node` we will be using. Notice the differences between `AstarNode` and `GrassFireNode`: this is why we are coupling graph creation with graph search!<sup>[1]</sup>

In [3]:
from typing import Optional
from graph import Node
import heapq


class AstarNode(Node):
    def __init__(self, x: int, y: int, cost_to_go: float) -> None:
        super().__init__(x, y, cost_to_go)
        self.h = 0.0
        self.parent: Optional[AstarNode] = None

    @property
    def f(self):
        return self.h + self.cost
    
    def __lt__(self, other):
        return self.f < other.f if self.f != other.f else self.h < other.h

The `x`, `y`, and `cost_to_go` are passed to the base class. `h` is initialized to some placeholder value (could have chosen `inf` as well). The `cost` attribute of the base `Node` corresponds to the `g` function in A* parlance, hence its use in the definition of the `f` property. The `parent` attribute is used to actually construct our path: by recursively retrieving parents, one constructs a path. You will notice that the definition of the `__lt__` method is missing - which brings us to the next question.

## Question 2

Later in the algorithm, we will feed the `AstarNode` into a priority queue (more specifically, Python's `heapq`). This will allow us to retrieve efficiently the node with the lower cost `f`). To this aim, we decide to implement a comparison operator for the node class.

a. implement the `__lt__` function above (note: this defines strict inequality `<`, not `<=`)

b. (answer after finishing the book) Should we (if so, why?) treat the case where $f_a = f_b$ but with $h_a < h_b$?

**<font color='red'>Given the same cost `f`, we should prefer the node with the lower `h` / higher `g` cost.**    </font>

**<font color='red'>Demonstration: there would be two possible situations if we kept exploring both paths $a$ and $b$ until the end of the algorithm:**
    </font>

**<font color='red'>1. One of the two paths is (stricly) better than the other. This information is however not available when we have to choose between $a$ and $b$, meaning that, at this point, choosing any path is correct.
</font>**

**<font color='red'>2. Both paths have the same final cost (i.e. they are alternative paths). In this case, if the number of iterations left is increasing in the value $h$ (fair assumption for most heuristics), then the path with the lower $h$ value has less "effort" left to reach the destination.
  </font>**


# Heuristic

The A* algorithm needs a heuristic `h` to direct the search. This function's goal is to give a rough estimate of the cost from a node `a` to a node `b` (usually the goal node). To allow trying out different heuristics, we created an abstract `AstarHeuristic` class:

In [4]:
from abc import ABC, abstractmethod

class AstarHeuristic(ABC):
    @abstractmethod
    def __call__(self, xs: int, ys: int, xf: int, yf: int) -> float:
        pass

Where `xs`, `ys` and `xf`, `yf` are the coordinates of the start and end node respectively.


## Question 3

The novel idea A* brought was the use of a heuristic. From this function's properties can various properties of A* be derived. Namely, an *optimistic* heuristic guarantees the algorithm's output will be optimal. Here, *optimistic* means the value given by the function is a lower-bound to the real cost of going from one node to another, i.e.:

$$
  h~\text{is optimistic } \Leftrightarrow h(n_a, n_b) \le g^*(n_a, n_b), \forall n_a, n_b \in \mathcal{N}
$$

Where $g^*(a, b)$ is the function giving the optimal cost for going from $a$ to $b$.


a. Implement the heuristic below

In [5]:
class Norm1AstarHeuristic(AstarHeuristic):
    def __call__(self, xs: int, ys: int, xf: int, yf: int) -> float:
        return abs(xs - xf) + abs(ys - yf)

Looking at the `grass_fire.py` code, you may have noticed we associate to cost-to-go (i.e. the cost of going from a node A to a node B) to the destination node (i.e. we do not explictly create the edges). This association is done by passing a `cost_map` $c : (\mathbb{R} \times \mathbb{R}) \rightarrow  \mathbb{R}$ to the constructor of the graph.
This map returns the cost-to-go associated to the node (identified by its $(x, y)$ coordinates).

We do the same with our `AstarGraph`, except we create `AstarNode`s instead of `GrassFireNode`s. The rest of the implementation is missing some bits. These you have to write yourself.

In [6]:
from graph import Graph
from typing import List
from math import inf
from logging import info, warning

class AstarGraph(Graph[AstarNode]):
    def __init__(self, max_x: int, max_y: int, cost_map, h: AstarHeuristic) -> None:
        super().__init__(max_x, max_y)
        self.h = h
        for z in range(max_x * max_y):
            x, y = self._unflatten(z)
            self._nodes.append(AstarNode(x, y, cost_map[(x, y)]))
            
    def reset_nodes(self):
        for n in self._nodes:
            n.cost = inf
            n.parent = None

    def collect_path(self, node: AstarNode) -> List[AstarNode]:
        path = [node]
        while path[-1].parent is not None:
            path.append(path[-1].parent)
        path.reverse()
        return path
    
    def get_neighbors(self, node: AstarNode, make_8_connected=True) -> List[AstarNode]:
        out = []
        du = [
            (-1, 0),
            (1, 0),
            (0, -1),
            (0, 1),
        ]
        
        if make_8_connected:
            du += [
                (1, 1),
                (1, -1),
                (-1, 1),
                (-1, -1),
            ]
            
        for (dx, dy) in du:
            if dx == 0 and dy == 0:
                continue
            next_node = self.try_node(node.x + dx, node.y + dy)
            if next_node:
                out.append(next_node)
        return out

    def compute_path(self, xs: int, ys: int, xf: int, yf: int) -> List[AstarNode]:
        self.reset_nodes()
        start = self.node(xs, ys)
        start.cost = 0 # cost from start to itself is obviously 0
        heap = [start]

        max_iter = 1000000
        for n in range(max_iter):
            try:
                node_to_explore = heapq.heappop(heap)
            except IndexError:
                warning(
                    f"No path exists from start ({xs}, {ys}) to goal ({xf}, {yf}) after {n+1} iterations"
                )
                return []

            x = node_to_explore.x
            y = node_to_explore.y

            if x == xf and y == yf:
                info(
                    f"Found path from start ({xs}, {ys}) to goal ({xf}, {yf}) after {n+1} iterations and {n+1+len(heap)} nodes added"
                )
                return self.collect_path(node_to_explore)

            for next_node in self.get_neighbors(node_to_explore, False):
                # The cost to go is the Euclidian distance crossed in each node,
                # multiplied by the nodes' cost-to-go
                # We do Euclidian in case the grid is 8-connected (see get_neighbors function),
                # but for a 4-connected grid this changes nothing
                dx = next_node.x - node_to_explore.x
                dy = next_node.y - node_to_explore.y
                dg = (
                    (dx**2 + dy**2) ** 0.5
                    * (next_node.cost_to_go + node_to_explore.cost_to_go)
                    * 0.5
                )
                g = node_to_explore.cost + dg

                # Already had a better path to reach this node, do nothing
                if next_node.cost <= g:
                    continue

                next_node.cost = g
                next_node.parent = node_to_explore
                h = self.h(next_node.x, next_node.y, xf, yf)
                next_node.h = h
                heapq.heappush(heap, next_node)

        warning(f"Could not find a path after {max_iter} iterations")
        return []

a. Why do we check `x == xf and y == yf` after popping a node from the heap, rather than before inserting it (i.e. inside the `for` loop)?

**<font color='red'>
When we add the goal node to the heap, we compute the cost of the complete path w.r.t the direct parent, i.e. `node_to_explore`. However, `node_to_explore` was selected partially based on its heuristic $h$. There might still be other nodes with a cheaper cost if the real cost from `node_to_explore` to the goal node is much higher than what the heuristic estimated (in practice this rarely matters).
</font>**


# Trying out the solution

Let's try the same thing we did with the `GrassFire`, but with our new A*. This is a trivial test (since there are no obstacles), but gives us a good idea of the best-case complexity:

In [7]:
h = Norm1AstarHeuristic()
a = AstarGraph
jm.run_astar_with_heuristic({'grid_size': 5, 'number_obstacles': 0, 'no_render': True}, a, h)
jm.run_astar_with_heuristic({'grid_size': 10, 'number_obstacles': 0, 'no_render': True}, a, h)
jm.run_astar_with_heuristic({'grid_size': 20, 'number_obstacles': 0, 'no_render': True}, a, h)
jm.run_astar_with_heuristic({'grid_size': 50, 'number_obstacles': 0, 'no_render': True}, a, h)
jm.run_astar_with_heuristic({'grid_size': 100, 'number_obstacles': 0, 'no_render': True}, a, h)
jm.run_astar_with_heuristic({'grid_size': 200, 'number_obstacles': 0, 'no_render': True}, a, h)
jm.run_astar_with_heuristic({'grid_size': 400, 'number_obstacles': 0, 'no_render': True}, a, h)
jm.run_astar_with_heuristic({'grid_size': 500, 'number_obstacles': 0, 'no_render': True}, a, h)

Path search took 0.17143999866675586ms, final cost: 8.0
Path search took 0.30127399804769084ms, final cost: 18.0
Path search took 0.9528299997327849ms, final cost: 38.0
Path search took 4.206935002002865ms, final cost: 98.0
Path search took 16.0021899973799ms, final cost: 198.0
Path search took 73.55937399916002ms, final cost: 398.0
Path search took 316.0085900017293ms, final cost: 798.0
Path search took 508.67706300050486ms, final cost: 998.0


Now let us try out real scenarios. First let's take a look at a map<sup>[2]</sup>

In [8]:
map_config = {'grid_size': 50, 'number_obstacles': 20, 'max_obstacle_size': 20}
import jupy_main as jm
jm.render_graph(map_config)

Some explanations about what you are now seeing:

1. The colors represent the cost of crossing the cells. By default, all cells are bright-green (i.e. unit cost).

2. Rectangular obstacles are generated (controlled by the `number_obstacles` parameter). The rectangles are not rotated, and the sides are uniformely samples from the range [1; `max_obstacle_size`[.

3. Black obstacles are not crossable at all (i.e. infinite cost), while other obstacles' color represent how "costly" it is cross the area (think mud, gravel, Belgian roads etc.). The cost is uniformely sampled inside [2; 10]. Bright red has cost 10, greenish is closer to 2.

4. The map is generated completely naïvely, meaning a path from the top-left to the bottom-right corners does not necessarily exist. Don't hesitate to visualize the map you are using if your algorithm fails to find a solution (the RNG is reset for each map so reproductibility is no issue<sup>[3]</sup>).

5. Because the generation is naive, obstacles can overlap etc.

6. `grid_size` is the default value for the width/height, but they can both be specified manually through `max_x`/`max_y`.
Now let us try to go from the top-left corner to the bottom-right one with our A* algorithm.

In [9]:
jm.run_astar_with_heuristic(map_config, a, h)

Path search took 32.408930001111ms, final cost: 98.0


Now let's try different map sizes.

In [14]:
jm.run_astar_with_heuristic({'grid_size': 100, 'number_obstacles': 40, 'max_obstacle_size': 20}, a, h)
jm.run_astar_with_heuristic({'grid_size': 200, 'number_obstacles': 60, 'max_obstacle_size': 25}, a, h)
jm.run_astar_with_heuristic({'grid_size': 300, 'number_obstacles': 100, 'max_obstacle_size': 40}, a, h)
jm.run_astar_with_heuristic({'grid_size': 500, 'number_obstacles': 300, 'max_obstacle_size': 50}, a, h)
jm.run_astar_with_heuristic({'grid_size': 1000, 'number_obstacles': 250, 'max_obstacle_size': 35}, a, h)
jm.run_astar_with_heuristic({'max_x': 2000, 'max_y': 1000, 'xf': 999, 'number_obstacles': 250, 'max_obstacle_size': 40}, a, h)

# Too big for my screen!
jm.run_astar_with_heuristic({'max_x': 2000, 'max_y': 1000, 'number_obstacles': 250, 'max_obstacle_size': 40}, a, h)


Path search took 172.8325740004948ms, final cost: 217.7728018049115
Path search took 674.0431459984393ms, final cost: 441.5001639708053
Path search took 1307.0763260002423ms, final cost: 823.4099405759927
Path search took 2262.5289850002446ms, final cost: 1221.9961854542557
Path search took 4462.769773999753ms, final cost: 1998.0
Path search took 4355.809816999681ms, final cost: 1998.0
Path search took 4669.68739100048ms, final cost: 2998.0


## Question 4

For these questions, trying out different map is *heavily* recommended.

a. How does the search scale w.r.t to $n = m^2$ (i.e. the square of the grid size)?

<font color='blue'>
TODO: last two should be more or less linear w.r.t runtime but they are basically equal, why? (oups)
</font>

b. Can you explain why the runtime does **not** seem directly related to the grid size?

**<font color='red'>The core idea behind A* is to use a heuristic. This heuristic directs a search, and, assuming it is "good enough", the complexity of the algorithm *does not depend on the input, but on its output*. For example, if you take a 1000\*1000 grid, make it 2000\*1000, and keep the goal the same. *your runtime will be virtually the same as the same path will be computed*<sup>[4]</sup>. An important implication of this is that A* works on infinite graphs. </font>**


## Question 5

Implement and try an L2 heuristic.

a. Are the final costs different? Explain why they are, or how they could be (*hint: this is related to how you connect neighboring nodes*).

**<font color='red'>
If the neighbors returned by `get_neighbors` are 4-connected: final cost will be the same, as the heuristic is still optimistic.
If the neighbors are 8-connected, then the cost will be lower (not necessarily strictly) since the previous (L1) heuristic was not optimistic, but the L2 is. With the L1 heuristic, cost for moving diagonally was over-estimated.
</font>**

b. Are the runtimes different? Explain why they are, or how they could be (*hint: this is related to how you connect neighboring nodes*).


**<font color='red'>
If the neighbors returned by `get_neighbors` are 4-connected: runtime will be higher. Both heuristics are optimistic (they are both lower bounds w.r.t real cost), but the L1 heuristic is a greater lower-bound, meaning fewer extra-nodes should be visited.
If the neighbors are 8-connected, then the cost will be *higher* since the previous (L1) heuristic was not optimistic, but the L2 is. An optimistic heuristic guarantees an optimal results, by forcing the algorithm the try every branch that has a chance to be better than other ones. A pessimistic heuristic means we might ignore (possibly) better paths and keep exploring (concrete) paths. This generally results in faster (sometimes *much faster*) runtime.
</font>**


c. Can you imagine a generic method to compromise between final cost and runtime?


**<font color='red'>
Using the answer to b., we see that manually worsening the value of a heuristic can make it master while sacrificing optimality guarantee. Using what has previously been done, this can trivially be implemented:
</font>**

```python
class WeightedAstarHeuristic(AstarHeuristic):
    def __init__(self, eps: float, base_h: AstarHeuristic) -> None:
        super().__init__()
        self.h = base_h
        self.eps = eps

    def __call__(self, xs: int, ys: int, xf: int, yf: int) -> float:
        return (1.0 + self.eps) * self.h(xs, ys, xf, yf)
```

**<font color='red'>
We pass to this Weighted A\*<sup>[5]</sup> heuristic another heuristic as well as a weighting factor. When the value `h` is computed, we take the value of the (potentially optimistic) underlying heuristic, and make it worse by multiplying by the weighting factor ($\epsilon >= 0$). For $\epsilon = 0$, this is equivalent to using the underlying heuristic directly.
</font>**

#### Footnotes & references

[1] A graph and a graph search algorithm are two different things, and the latter can be implemented generically over the former. This requires however the Graph (as well as the Node and Edge) class to implement a number of functions. While not a specifically daunting task, this is not within the scope of this exercise. We have therefore decided to keep the graph creation and search tighly coupled, allowing you (almost) complete freedom over the way nodes and edges are stored. For an example implementation of a generic graph library, the Boost Graph Library (BGL) is your best bet: https://www.boost.org/doc/libs/1_81_0/libs/graph/doc/index.html

[2] This requires you to run this file locally and with a working pygame installation. If this is not possible, change all the `no_render` boolean values to `True`. You can still see the runtime information of the algorithm, but not the actual result, so try to get this running somehow :) if anyone knows of a nice alternative for rendering things in Jupyter, I *am* all ears.

[3] the values for the maps I'm generating here should always work. Well, at least, according to a famous search engine, the numpy RNG (used here because the stdlib's is terribly slow) should be consistent accross platforms (Windows / OSX / Linux). Meaning we should both see the same map. If any of them does not work, fiddle a bit with the values until it does.

[4] Well, not necessarily, we could imagine a path with a very high cost in the 1000\*1000 grid that can be avoided in the 2000\*1000 but you got the point I think.

[5] Pohl I. Heuristic search viewed as path finding in a graph. Artificial intelligence. 1970 Jan 1;1(3-4):193-204.